In [70]:
import csv
import numpy as np
from sklearn.metrics import log_loss
import torch
from torch import autograd,nn, optim
import torch.nn.functional as F

In [2]:
year = 2005
with open('data-'+ str(year) +'/game.csv', 'r') as f:
    reader = csv.reader(f)
    Game = list(reader)
Game = Game[1:]

with open('data-' + str(year) + '/game-statistics.csv', 'r') as f:
    reader = csv.reader(f)
    GameStatistics = list(reader)
    GameStatistics = GameStatistics[1:]
    
with open('data-' + str(year) + '/team-game-statistics.csv', 'r') as f:
    reader = csv.reader(f)
    TeamGameStatistics = list(reader)
    TeamGameStatistics = TeamGameStatistics[1:]

    
#with open('data-2005/team-game-statistics.csv', 'r') as f:
 # reader = csv.reader(f)
  #TeamGameStatistics = list(reader)

In [3]:
#Assuming no anomolies in game data in TEAMGAMESTATS correspnoding to games in GAME
#Add fields from TEAMSGAMESTATS in this function
def SearchGame(GameID, TeamGameStatistics, HomeTeam, VisitTeam):
    count = 0
    for i,gamestats in enumerate(TeamGameStatistics):
        if(gamestats[1] == GameID):
            if(gamestats[0] == HomeTeam):
                HomeTeamStats = gamestats
            else:
                VisitTeamStats = gamestats
            count = count + 1
        if(count == 2):
            return(HomeTeamStats,VisitTeamStats)
    print("Loop ended without returning! :( ")

In [4]:
def listOfDictionaries(GameList, Game, GameStatistics, TeamGameStatistics):
    newGame = {}
    newGame['Date'] = Game[1]
    newGame['GameID'] = Game[0]
    newGame['Attendance'] = GameStatistics[1]
    newGame['Duration'] = GameStatistics[2]
    newGame['HomeTeam'] = int(Game[3])
    newGame['VisitTeam'] = int(Game[2])
    HomeTeamStats, VisitTeamStats = SearchGame(Game[0],TeamGameStatistics,Game[3],Game[2])
    newGame['HT-Points'] = int(HomeTeamStats[35])
    newGame['VT-Points'] = int(VisitTeamStats[35])
    GameList.append(newGame)
    return GameList

In [5]:
GameList = [] #List of dictionaries, based on sorted order of games
TeamList = [] #List of Teams in sorted order of Team codes
count = 0
for i in range(len(Game)):
    GameList = listOfDictionaries(GameList, Game[i], GameStatistics[i], TeamGameStatistics)
    GameList
    if(GameList[i]['HomeTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['HomeTeam'])
    if(GameList[i]['VisitTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['VisitTeam'])
        
TeamList.sort()

In [6]:
GameList[0]

{'Attendance': '40000',
 'Date': '09/01/2005',
 'Duration': '164',
 'GameID': '0086016420050901',
 'HT-Points': 38,
 'HomeTeam': 164,
 'VT-Points': 0,
 'VisitTeam': 86}

In [7]:
NumberOfWins = []
NumberOfMatches = []
for i,team in enumerate(TeamList):
    NumberOfWins.append(0)
    NumberOfMatches.append(0)

In [11]:
#win ratios of home team, win ratio of visit team
X_train = []
X_test = []
#1 if home team wins, zero otherwise
Y_train = []
Y_test = []

In [9]:
#Skip first 100 games
gamecount = 0
for i,game in enumerate(GameList[:100]):
    gamecount = gamecount + 1
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    if(game['HT-Points'] >= game['VT-Points']):
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1        
    

In [10]:
for i,team in enumerate(TeamList):
    if(NumberOfMatches[i] < NumberOfWins[i]):
        print("Matches: ", NumberOfMatches[i], " Wins: ", NumberOfWins[i])

In [13]:
gamecount = 0
for i,game in enumerate(GameList[100:]):
    temp = []
    gamecount = gamecount + 1
    print("For ",i,": ")
    #print(game)
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    print(NumberOfWins[htindex])
    print(NumberOfMatches[htindex])
    print(NumberOfWins[vtindex])
    print(NumberOfMatches[vtindex])
    if(NumberOfMatches[htindex] > 0):
        temp.append(NumberOfWins[htindex]/NumberOfMatches[htindex])
    else:
        temp.append(0.0)
    if(NumberOfMatches[vtindex] > 0):
        temp.append(NumberOfWins[vtindex]/NumberOfMatches[vtindex])
    else:
        temp.append(0.0)
    X_train.append(temp)
    if(game['HT-Points'] >= game['VT-Points']):
        Y_train.append(1)
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        Y_train.append(0)
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1

For  0 : 
1
1
1
1
For  1 : 
0
1
0
0
For  2 : 
1
1
0
0
For  3 : 
0
1
0
1
For  4 : 
0
1
0
0
For  5 : 
1
1
0
1
For  6 : 
0
1
0
0
For  7 : 
0
1
1
1
For  8 : 
1
1
1
1
For  9 : 
0
1
0
1
For  10 : 
1
1
0
0
For  11 : 
1
1
0
0
For  12 : 
1
1
1
1
For  13 : 
0
1
0
0
For  14 : 
0
1
1
1
For  15 : 
1
1
0
1
For  16 : 
0
0
0
0
For  17 : 
1
1
0
1
For  18 : 
1
1
0
0
For  19 : 
1
2
2
2
For  20 : 
1
1
1
2
For  21 : 
0
2
0
1
For  22 : 
1
2
2
2
For  23 : 
2
2
1
2
For  24 : 
1
1
1
2
For  25 : 
1
2
0
2
For  26 : 
0
1
2
2
For  27 : 
2
2
1
1
For  28 : 
1
1
0
0
For  29 : 
1
2
0
1
For  30 : 
2
2
1
2
For  31 : 
2
2
2
2
For  32 : 
0
2
1
1
For  33 : 
0
1
0
0
For  34 : 
1
2
1
2
For  35 : 
2
2
0
2
For  36 : 
2
2
2
2
For  37 : 
2
2
0
0
For  38 : 
0
2
0
2
For  39 : 
2
2
2
2
For  40 : 
1
2
0
0
For  41 : 
2
2
1
2
For  42 : 
2
2
0
1
For  43 : 
0
1
0
0
For  44 : 
1
2
0
2
For  45 : 
2
2
0
1
For  46 : 
2
2
2
2
For  47 : 
0
0
1
2
For  48 : 
2
2
1
1
For  49 : 
0
1
1
2
For  50 : 
2
2
0
2
For  51 : 
2
2
0
2
For  52 : 
1
2
0
0
For

7
5
8
For  435 : 
3
9
1
7
For  436 : 
6
8
4
8
For  437 : 
5
7
4
9
For  438 : 
3
8
5
8
For  439 : 
8
9
8
9
For  440 : 
4
8
4
8
For  441 : 
4
8
4
7
For  442 : 
4
8
7
8
For  443 : 
7
8
7
9
For  444 : 
6
8
6
9
For  445 : 
8
9
5
9
For  446 : 
3
8
2
8
For  447 : 
7
8
7
9
For  448 : 
3
9
3
9
For  449 : 
5
9
4
9
For  450 : 
4
9
5
9
For  451 : 
1
9
0
9
For  452 : 
4
8
6
9
For  453 : 
6
9
5
9
For  454 : 
6
9
7
9
For  455 : 
4
9
4
8
For  456 : 
5
9
3
8
For  457 : 
4
9
2
7
For  458 : 
5
9
7
9
For  459 : 
6
9
7
9
For  460 : 
5
8
6
8
For  461 : 
3
9
2
9
For  462 : 
6
9
4
9
For  463 : 
8
10
5
9
For  464 : 
5
9
4
9
For  465 : 
9
9
5
9
For  466 : 
4
9
2
8
For  467 : 
9
9
7
8
For  468 : 
2
8
5
8
For  469 : 
4
8
4
8
For  470 : 
2
8
0
0
For  471 : 
3
8
4
8
For  472 : 
4
10
7
8
For  473 : 
6
9
5
9
For  474 : 
9
10
2
9
For  475 : 
0
9
5
8
For  476 : 
5
9
5
9
For  477 : 
6
9
4
8
For  478 : 
1
9
2
8
For  479 : 
3
7
3
8
For  480 : 
7
9
6
9
For  481 : 
3
9
8
9
For  482 : 
5
9
3
9
For  483 : 
1
8
4
7
For  484 : 

In [24]:
X_train

[[1.0, 1.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [0.0, 1.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.5, 1.0],
 [1.0, 0.5],
 [0.0, 0.0],
 [0.5, 1.0],
 [1.0, 0.5],
 [1.0, 0.5],
 [0.5, 0.0],
 [0.0, 1.0],
 [1.0, 1.0],
 [1.0, 0.0],
 [0.5, 0.0],
 [1.0, 0.5],
 [1.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.0],
 [0.5, 0.5],
 [1.0, 0.0],
 [1.0, 1.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.5, 0.0],
 [1.0, 0.5],
 [1.0, 0.0],
 [0.0, 0.0],
 [0.5, 0.0],
 [1.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.5],
 [1.0, 1.0],
 [0.0, 0.5],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.5, 0.0],
 [0.0, 1.0],
 [0.5, 1.0],
 [1.0, 0.5],
 [1.0, 0.5],
 [1.0, 1.0],
 [1.0, 0.0],
 [0.5, 1.0],
 [1.0, 0.0],
 [0.0, 0.5],
 [1.0, 0.0],
 [0.5, 0.0],
 [0.0, 0.0],
 [0.0, 0.5],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 1.0],
 [0.5, 0.5],
 [1.0, 0.0],
 [0.0, 0.0],
 [0.5, 1.0],
 [0.5, 1.0],
 [0.5, 0.0],
 [0.0, 1.0],

In [55]:
batch_size = gamecount
input_size = 2
hidden_size = 5
num_classes = 2
learning_rate = 0.0001

#input = autograd.Variable(torch.rand(batch_size,input_size)
input = autograd.Variable(torch.Tensor(X_train))

#input[0][1] = 2
target = autograd.Variable(torch.Tensor(Y_train).long())

In [64]:
i = 576
print(X_train[i])
print(input[i])
print(target[i])
print(Y_train[i])

[0.0, 0.2]
Variable containing:
 0.0000
 0.2000
[torch.FloatTensor of size 2]

Variable containing:
 0
[torch.LongTensor of size 1]

0


In [57]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size,num_classes):
        super().__init__()
        #create hidden layers and assign it to self
        self.h1 = nn.Linear(input_size,hidden_size)
        self.h2 = nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        #run forward propogation
        x = self.h1(x)
        x = F.tanh(x)
        x = self.h2(x)
        x = F.softmax(x)
        return x

In [58]:
model = Net(input_size = input_size, hidden_size = hidden_size, num_classes = num_classes)
opt = optim.Adam(params=model.parameters(),lr = learning_rate)

In [59]:
for epoch in range(1000):
    out = model(input)
    _, pred = out.max(1)
   # print("Target: ",str(target.view(1,-1)).split('\n')[1])
   # print("Pred: ", str(pred.view(1,-1)).split('\n')[1])
    loss = F.nll_loss(out,target)
    print("Loss: ",loss.data[0])
    model.zero_grad()
    loss.backward()
    opt.step()

C:\Users\Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Target:  
Pred:  
Loss:  -0.476009726524353
Target:  
Pred:  
Loss:  -0.4760412275791168
Target:  
Pred:  
Loss:  -0.4760722815990448
Target:  
Pred:  
Loss:  -0.4761035442352295
Target:  
Pred:  
Loss:  -0.47613486647605896
Target:  
Pred:  
Loss:  -0.4761664569377899
Target:  
Pred:  
Loss:  -0.47619733214378357
Target:  
Pred:  
Loss:  -0.4762289226055145
Target:  
Pred:  
Loss:  -0.4762600362300873
Target:  
Pred:  
Loss:  -0.4762912392616272
Target:  
Pred:  
Loss:  -0.47632265090942383
Target:  
Pred:  
Loss:  -0.4763539731502533
Target:  
Pred:  
Loss:  -0.47638535499572754
Target:  
Pred:  
Loss:  -0.47641661763191223
Target:  
Pred:  
Loss:  -0.476448118686676
Target:  
Pred:  
Loss:  -0.47647953033447266
Target:  
Pred:  
Loss:  -0.47651100158691406
Target:  
Pred:  
Loss:  -0.476542204618454
Target:  
Pred:  
Loss:  -0.4765734076499939
Target:  
Pred:  
Loss:  -0.4766048789024353
Target:  
Pred:  
Loss:  -0.47663623094558716
Target:  
Pred:  
Loss:  -0.47666749358177185
Targ

Target:  
Pred:  
Loss:  -0.481881707906723
Target:  
Pred:  
Loss:  -0.4819137454032898
Target:  
Pred:  
Loss:  -0.4819459021091461
Target:  
Pred:  
Loss:  -0.481978178024292
Target:  
Pred:  
Loss:  -0.48201048374176025
Target:  
Pred:  
Loss:  -0.4820426404476166
Target:  
Pred:  
Loss:  -0.48207467794418335
Target:  
Pred:  
Loss:  -0.48210716247558594
Target:  
Pred:  
Loss:  -0.48213931918144226
Target:  
Pred:  
Loss:  -0.4821716547012329
Target:  
Pred:  
Loss:  -0.48220396041870117
Target:  
Pred:  
Loss:  -0.48223599791526794
Target:  
Pred:  
Loss:  -0.4822680950164795
Target:  
Pred:  
Loss:  -0.4823005497455597
Target:  
Pred:  
Loss:  -0.48233234882354736
Target:  
Pred:  
Loss:  -0.4823647439479828
Target:  
Pred:  
Loss:  -0.4823969900608063
Target:  
Pred:  
Loss:  -0.4824290871620178
Target:  
Pred:  
Loss:  -0.4824615716934204
Target:  
Pred:  
Loss:  -0.48249366879463196
Target:  
Pred:  
Loss:  -0.4825262129306793
Target:  
Pred:  
Loss:  -0.4825585186481476
Targ

Target:  
Pred:  
Loss:  -0.4881199598312378
Target:  
Pred:  
Loss:  -0.4881535470485687
Target:  
Pred:  
Loss:  -0.4881865978240967
Target:  
Pred:  
Loss:  -0.48821985721588135
Target:  
Pred:  
Loss:  -0.48825329542160034
Target:  
Pred:  
Loss:  -0.48828673362731934
Target:  
Pred:  
Loss:  -0.4883199632167816
Target:  
Pred:  
Loss:  -0.48835286498069763
Target:  
Pred:  
Loss:  -0.48838669061660767
Target:  
Pred:  
Loss:  -0.4884195625782013
Target:  
Pred:  
Loss:  -0.488453209400177
Target:  
Pred:  
Loss:  -0.4884866178035736
Target:  
Pred:  
Loss:  -0.4885197579860687
Target:  
Pred:  
Loss:  -0.4885529577732086
Target:  
Pred:  
Loss:  -0.4885864853858948
Target:  
Pred:  
Loss:  -0.4886196553707123
Target:  
Pred:  
Loss:  -0.48865318298339844
Target:  
Pred:  
Loss:  -0.48868653178215027
Target:  
Pred:  
Loss:  -0.48872023820877075
Target:  
Pred:  
Loss:  -0.48875343799591064
Target:  
Pred:  
Loss:  -0.4887867569923401
Target:  
Pred:  
Loss:  -0.4888205826282501
Ta

Pred:  
Loss:  -0.4950753152370453
Target:  
Pred:  
Loss:  -0.4951103627681732
Target:  
Pred:  
Loss:  -0.4951449930667877
Target:  
Pred:  
Loss:  -0.49518001079559326
Target:  
Pred:  
Loss:  -0.49521520733833313
Target:  
Pred:  
Loss:  -0.4952497184276581
Target:  
Pred:  
Loss:  -0.4952847957611084
Target:  
Pred:  
Loss:  -0.49531951546669006
Target:  
Pred:  
Loss:  -0.495354562997818
Target:  
Pred:  
Loss:  -0.49538907408714294
Target:  
Pred:  
Loss:  -0.49542438983917236
Target:  
Pred:  
Loss:  -0.4954591989517212
Target:  
Pred:  
Loss:  -0.49549436569213867
Target:  
Pred:  
Loss:  -0.4955288767814636
Target:  
Pred:  
Loss:  -0.495563805103302
Target:  
Pred:  
Loss:  -0.49559906125068665
Target:  
Pred:  
Loss:  -0.495633602142334
Target:  
Pred:  
Loss:  -0.495668888092041
Target:  
Pred:  
Loss:  -0.4957033395767212
Target:  
Pred:  
Loss:  -0.495738685131073
Target:  
Pred:  
Loss:  -0.4957733154296875
Target:  
Pred:  
Loss:  -0.49580878019332886
Target:  
Pred:  

Target:  
Pred:  
Loss:  -0.5025428533554077
Target:  
Pred:  
Loss:  -0.5025780200958252
Target:  
Pred:  
Loss:  -0.502613365650177
Target:  
Pred:  
Loss:  -0.5026493072509766
Target:  
Pred:  
Loss:  -0.5026840567588806
Target:  
Pred:  
Loss:  -0.5027194619178772
Target:  
Pred:  
Loss:  -0.5027546882629395
Target:  
Pred:  
Loss:  -0.5027898550033569
Target:  
Pred:  
Loss:  -0.5028250217437744
Target:  
Pred:  
Loss:  -0.5028606653213501
Target:  
Pred:  
Loss:  -0.5028960704803467
Target:  
Pred:  
Loss:  -0.5029314756393433
Target:  
Pred:  
Loss:  -0.5029668807983398
Target:  
Pred:  
Loss:  -0.503002405166626
Target:  
Pred:  
Loss:  -0.5030374526977539
Target:  
Pred:  
Loss:  -0.5030726194381714
Target:  
Pred:  
Loss:  -0.503108024597168
Target:  
Pred:  
Loss:  -0.5031431913375854
Target:  
Pred:  
Loss:  -0.5031787157058716
Target:  
Pred:  
Loss:  -0.5032138824462891
Target:  
Pred:  
Loss:  -0.5032492280006409
Target:  
Pred:  
Loss:  -0.5032846927642822
Target:  
Pre

Loss:  -0.509063720703125
Target:  
Pred:  
Loss:  -0.50909823179245
Target:  
Pred:  
Loss:  -0.5091329216957092
Target:  
Pred:  
Loss:  -0.5091676115989685
Target:  
Pred:  
Loss:  -0.5092021822929382
Target:  
Pred:  
Loss:  -0.5092364549636841
Target:  
Pred:  
Loss:  -0.5092714428901672
Target:  
Pred:  
Loss:  -0.5093058943748474
Target:  
Pred:  
Loss:  -0.5093402862548828
Target:  
Pred:  
Loss:  -0.5093750357627869
Target:  
Pred:  
Loss:  -0.5094095468521118
Target:  
Pred:  
Loss:  -0.5094441175460815
Target:  
Pred:  
Loss:  -0.5094784498214722
Target:  
Pred:  
Loss:  -0.5095129609107971
Target:  
Pred:  
Loss:  -0.5095475912094116
Target:  
Pred:  
Loss:  -0.5095820426940918
Target:  
Pred:  
Loss:  -0.5096164345741272
Target:  
Pred:  
Loss:  -0.5096510052680969
Target:  
Pred:  
Loss:  -0.5096856355667114
Target:  
Pred:  
Loss:  -0.5097202658653259
Target:  
Pred:  
Loss:  -0.5097548365592957
Target:  
Pred:  
Loss:  -0.5097890496253967


In [60]:
print("Target: ",str(target.view(1,-1)).split('\n')[1])
print("Pred: ", str(pred.view(1,-1)).split('\n')[1])

Target:  
Pred:  


In [65]:
pred

Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

In [66]:
target

Variable containing:
 1
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 1
 0
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 1
 1
 1
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 1
 1
 0
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 1
 0
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 1
 0
 1
 0
 1
 0
 1
 0
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 1
 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 0
 1
 1
 1
 1
 0
 1
 1
 1
 0
 0
 1
 1
 1
 1
 1
 1
 0
 0
 1
 0
 0
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 0
 1
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 0
 0
 

In [67]:
def Accuracy(Y_train,Yhat):
    CorrectPredictions = 0
    print(len(Y_train))
    print(len(Yhat))
    for i,current in enumerate(Y_train):
        if((Y_train[i] == 0 and Yhat[i] < 0.5) or (Y_train[i] == 1 and Yhat[i] > 0.5)):
            CorrectPredictions = CorrectPredictions + 1
    return(100 * CorrectPredictions/len(Y_train), CorrectPredictions)


In [73]:
t = target.data.numpy()

In [74]:
accuracy, CorrectPredictions = Accuracy(t,pred.data.numpy())

618
618


In [75]:
CorrectPredictions

359

In [76]:
accuracy

58.090614886731395